# UF Research Computing  

![UF Research Computing Logo](images/ufrc_logo.png)


This tutorial is adapted from: [Understanding PyTorch with an example: a step-by-step tutorial](https://towardsdatascience.com/understanding-pytorch-with-an-example-a-step-by-step-tutorial-81fc5f8c4e8e) by Daniel Godoy.


## Optimizer

So far, we’ve been **manually** updating the parameters using the computed gradients. That’s probably fine for *two parameters*… but what if we had a whole lot of them?! We use one of PyTorch’s **optimizers**, like [**SGD**](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD) or [**Adam**](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam).

An optimizer takes the **parameters** we want to update, the **learning rate** we want to use (and possibly many other hyper-parameters as well!) and **performs the updates** through its [`step()`](https://pytorch.org/docs/stable/optim.html#torch.optim.Optimizer.step) method.

Besides, we also don’t need to zero the gradients one by one anymore. We just invoke the optimizer’s [`zero_grad()`](https://pytorch.org/docs/stable/optim.html#torch.optim.Optimizer.zero_grad) method and that’s it!

In the code below, we create a *Stochastic Gradient Descent* (SGD) optimizer to update our parameters **a** and **b**.

<div class="alert alert-info"><b>Note</b><p>
    Don’t be fooled by the optimizer’s name: if we use <b>all training data</b> at once for the update — as we are actually doing in the code — the optimizer is performing a <b>batch gradient descent</b>, despite of its name.</p></div>

In [6]:
# Re-establish the environment
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import random

%run generate_data.py

In [7]:
torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
print(a, b)

lr = 1e-1
n_epochs = 1000

# Defines a SGD optimizer to update the parameters
optimizer = optim.SGD([a, b], lr=lr)


tensor([0.1940], device='cuda:0', requires_grad=True) tensor([0.1391], device='cuda:0', requires_grad=True)


In [8]:
for epoch in range(n_epochs):
    yhat = a + b * x_train_tensor
    error = y_train_tensor - yhat
    loss = (error ** 2).mean()

    loss.backward()    
    
    # No more manual update!
    # with torch.no_grad():
    #     a -= lr * a.grad
    #     b -= lr * b.grad
    optimizer.step()
    
    # No more telling PyTorch to let gradients go!
    # a.grad.zero_()
    # b.grad.zero_()
    optimizer.zero_grad()
    
print(a, b)

tensor([1.0235], device='cuda:0', requires_grad=True) tensor([1.9690], device='cuda:0', requires_grad=True)


Cool! We’ve optimized the **optimization** process :-) *What’s left?*

## Loss

We now tackle the **loss computation**. As expected, PyTorch got us covered once again. There are many [loss functions](https://pytorch.org/docs/stable/nn.html#loss-functions) to choose from, depending on the task at hand. Since ours is a regression, we are using the [Mean Square Error (MSE) loss](https://pytorch.org/docs/stable/nn.html#torch.nn.MSELoss).

Notice that `nn.MSELoss` actually **creates a loss function** for us — **it is NOT the loss function itself**. Moreover, you can specify a **reduction method** to be applied, that is, **how do you want to aggregate the results for individual points** — *you can average them (`reduction='mean'`) or simply sum them up (`reduction='sum'`).*

We then **use** the created loss function later, at line 20, to compute the loss given our **predictions** and our **labels**.

Our code looks like this now:

In [9]:
torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
print(a, b)

lr = 1e-1
n_epochs = 1000

# Defines a MSE loss function
loss_fn = nn.MSELoss(reduction='mean')

optimizer = optim.SGD([a, b], lr=lr)

for epoch in range(n_epochs):
    yhat = a + b * x_train_tensor
    
    # No more manual loss!
    # error = y_tensor - yhat
    # loss = (error ** 2).mean()
    loss = loss_fn(y_train_tensor, yhat)

    loss.backward()    
    optimizer.step()
    optimizer.zero_grad()
    
print(a, b)

tensor([0.1940], device='cuda:0', requires_grad=True) tensor([0.1391], device='cuda:0', requires_grad=True)
tensor([1.0235], device='cuda:0', requires_grad=True) tensor([1.9690], device='cuda:0', requires_grad=True)


At this point, there’s only one piece of code left to change: the **predictions**. It is then time to introduce PyTorch’s way of implementing a…

## Model

In PyTorch, a **model** is represented by a regular **Python class** that inherits from the [**Module**](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) class.

The most fundamental methods it needs to implement are:
* `__init__(self)`: **it defines the parts that make up the model** — in our case, *two parameters*, **a** and **b**.

  You are not limited to defining **parameters**, though… **models can contain other models (or layers) as its attributes** as well, so you can easily nest them. We’ll see an example of this shortly as well.

* [`forward(self, x)`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module.forward): it performs the **actual computation**, that is, it **outputs a prediction**, given the input **x**.

 You should **NOT call the `forward(x)`** method, though. You should **call the whole model itself**, as in `model(x)` to perform a forward pass and output predictions.

Let’s build a proper (yet simple) model for our regression task. It should look like this:

In [10]:
class ManualLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        # To make "a" and "b" real parameters of the model, we need to wrap them with nn.Parameter
        self.a = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        self.b = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        
    def forward(self, x):
        # Computes the outputs / predictions
        return self.a + self.b * x

In the `__init__` method, we define our **two parameters**, **a** and **b**, using the [`Parameter()`](https://pytorch.org/docs/stable/nn.html#torch.nn.Parameter) class, to tell PyTorch these **tensors should be considered parameters of the model they are an attribute of**.

Why should we care about that? By doing so, we can use our model’s [`parameters()`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module.parameters) method to retrieve **an iterator over all model’s parameters**, even those parameters of **nested models**, that we can use to feed our optimizer (instead of building a list of parameters ourselves!).

Moreover, we can get the **current values for all parameters** using our model’s [`state_dict()`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module.state_dict) method.

<div class="alert alert-info"><b>Important</b><p>
We need to send our model to the same device where the data is. If our data is made of GPU tensors, our model must “live” inside the GPU as well.</p></div>

We can use all these handy methods to change our code, which should be looking like this:

In [11]:
torch.manual_seed(42)

# Now we can create a model and send it at once to the device
model = ManualLinearRegression().to(device)
# We can also inspect its parameters using its state_dict
print(model.state_dict())

lr = 1e-1
n_epochs = 1000

loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=lr)

for epoch in range(n_epochs):
    # What is this?!?
    model.train()

    # No more manual prediction!
    # yhat = a + b * x_tensor
    yhat = model(x_train_tensor)
    
    loss = loss_fn(y_train_tensor, yhat)
    loss.backward()    
    optimizer.step()
    optimizer.zero_grad()
    
print(model.state_dict())

OrderedDict([('a', tensor([0.3367], device='cuda:0')), ('b', tensor([0.1288], device='cuda:0'))])
OrderedDict([('a', tensor([1.0235], device='cuda:0')), ('b', tensor([1.9690], device='cuda:0'))])


I hope you noticed one particular statement in the code, to which I assigned a comment **“What is this?!?”** — `model.train()`.

In PyTorch, models have a `train()` method which, somewhat disappointingly, **does NOT perform a training step**. Its only purpose is to **set the model to training mode**. Why is this important? Some models may use mechanisms like [**Dropout**](https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout), for instance, which have **distinct behaviors in training and evaluation** phases.

## Nested Models

In our model, we manually created two parameters to perform a linear regression. Let’s use PyTorch’s [**Linear**](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear) model as an attribute of our own, thus creating a nested model.

Even though this clearly is a contrived example, as we are pretty much wrapping the underlying model without adding anything useful (or, at all!) to it, it illustrates well the concept.

In the `__init__` method, we created an **attribute** that contains our **nested `Linear` model**.

In the `forward()` method, we **call the nested model itself** to perform the forward pass (*notice, we are **not** calling `self.linear.forward(x)`!*).

In [12]:
class LayerLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        # Instead of our custom parameters, we use a Linear layer with single input and single output
        self.linear = nn.Linear(1, 1)
                
    def forward(self, x):
        # Now it only takes a call to the layer to make predictions
        return self.linear(x)

Now, if we call the `parameters()` method of this model, **PyTorch will figure the parameters of its attributes in a recursive way**. You can try it yourself using something like: `[*LayerLinearRegression().parameters()]` to get a list of all parameters. You can also add new `Linear` attributes and, even if you don’t use them at all in the forward pass, they will **still** be listed under `parameters()`.

## Sequential Models

Our model was simple enough… You may be thinking: *"why even bother to build a class for it?!"* Well, you have a point…

For **straightforward models**, that use run-of-the-mill layers, where the output of a layer is sequentially fed as an input to the next, we can use a, er… [**Sequential**](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential) model :-)

In our case, we would build a Sequential model with a single argument, that is, the `Linear` layer we used to train our linear regression. The model would look like this:

In [13]:
# Alternatively, you can use a Sequential model
model = nn.Sequential(nn.Linear(1, 1)).to(device)

Simple enough, right?

## Training Step

So far, we’ve defined an **optimizer**, a **loss function** and a **model**. Scroll up a bit and take a quick look at the code *inside the loop*. Would it **change** if we were using a **different optimizer**, or **loss**, or even **model**? If not, how can we make it **more generic**?

Well, I guess we could say all these lines of code **perform a training step**, given those **three elements** (*optimizer, loss and model*),the **features** and the **labels**.

So, how about **writing a function that takes those three elements** and **returns another function that performs a training step**, taking a set of features and labels as arguments and returning the corresponding loss?

Then we can use this general-purpose function to build a `train_step()` function to be called inside our training loop. Now our code should look like this… see how **tiny** the training loop is now?


In [14]:
def make_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def train_step(x, y):
        # Sets model to TRAIN mode
        model.train()
        # Makes predictions
        yhat = model(x)
        # Computes loss
        loss = loss_fn(y, yhat)
        # Computes gradients
        loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        # Returns the loss
        return loss.item()
    
    # Returns the function that will be called inside the train loop
    return train_step

# Creates the train_step function for our model, loss function and optimizer
train_step = make_train_step(model, loss_fn, optimizer)
losses = []

# For each epoch...
for epoch in range(n_epochs):
    # Performs one train step and returns the corresponding loss
    loss = train_step(x_train_tensor, y_train_tensor)
    losses.append(loss)
    
# Checks model's parameters
print(model.state_dict())

OrderedDict([('0.weight', tensor([[-0.2191]], device='cuda:0')), ('0.bias', tensor([0.2018], device='cuda:0'))])


Let’s give our training loop a rest and focus on our **data** for a while… so far, we’ve simply used our *Numpy arrays* turned **PyTorch tensors**. But we can do better, we can build a dataset.

**[Continue to part 5 of the tutorial: 05_Dataset_DataLoader.ipynb](05_Dataset_DataLoader.ipynb)**